**Train Test and Cross Validation**

In [10]:
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

df = df.dropna()

X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']

scaler = StandardScaler()
X = scaler.fit_transform(X)

knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3,5,7,9,11,13,15],  
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(knn, param_grid, scoring='accuracy', cv=kf, n_jobs=-1, verbose=1)

grid_search.fit(X, y)

best_knn = grid_search.best_estimator_

y_pred = best_knn.predict(X)

print("Best Parameters for KNN:", grid_search.best_params_)
print("Mean Accuracy with Best Parameters:", grid_search.best_score_)

print("KNN Classification Report:")
print(classification_report(y, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))


Fitting 5 folds for each of 7 candidates, totalling 35 fits


Best Parameters for KNN: {'n_neighbors': 15}
Mean Accuracy with Best Parameters: 0.7322552697266438
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.70      0.75      1638
           1       0.73      0.82      0.78      1630

    accuracy                           0.76      3268
   macro avg       0.77      0.76      0.76      3268
weighted avg       0.77      0.76      0.76      3268

Confusion Matrix:
[[1148  490]
 [ 287 1343]]


In [11]:
from sklearn.metrics import classification_report, confusion_matrix

valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"

df_valid = pd.read_csv(valid_path)
df_valid = df_valid.dropna()

X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

X_valid_scaled = scaler.transform(X_valid)

print(f"🧪 KNN - Validation dataset shape: {X_valid.shape}")

y_pred_valid = best_knn.predict(X_valid_scaled)

print(f"📌 KNN - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

print("🟦 KNN - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


🧪 KNN - Validation dataset shape: (364, 13)
📌 KNN - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       178
           1       0.71      0.77      0.74       186

    accuracy                           0.72       364
   macro avg       0.72      0.72      0.72       364
weighted avg       0.72      0.72      0.72       364

🟦 KNN - Validation Confusion Matrix:
[[120  58]
 [ 43 143]]


17:

🧪 KNN - Validation dataset shape: (159, 10)
📌 KNN - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.59      0.66        76
           1       0.69      0.82      0.75        83

    accuracy                           0.71       159
   macro avg       0.72      0.71      0.70       159
weighted avg       0.72      0.71      0.71       159

🟦 KNN - Validation Confusion Matrix:
[[45 31]
 [15 68]]

15:

🧪 KNN - Validation dataset shape: (159, 10)
📌 KNN - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.58      0.65        76
           1       0.68      0.82      0.74        83

    accuracy                           0.70       159
   macro avg       0.71      0.70      0.70       159
weighted avg       0.71      0.70      0.70       159

🟦 KNN - Validation Confusion Matrix:
[[44 32]
 [15 68]]
